## 1. Tải Dữ Liệu từ CSV

In [33]:
!pip install datasets

In [34]:
from transformers import AutoTokenizer, TrainingArguments, Trainer, AutoModel
import numpy as np
import torch
from datasets import load_dataset
import torch.nn as nn
import os
from typing import List
from tqdm import tqdm


# os.environ["CUDA_VISIBLE_DEVICES"] = "1" ## Setup CUDA GPU 1



In [35]:
import torch

# Kiểm tra GPU khả dụng
print("CUDA available:", torch.cuda.is_available())
print("Number of GPUs:", torch.cuda.device_count())
if torch.cuda.is_available():
    for i in range(torch.cuda.device_count()):
        print(f"GPU {i}: {torch.cuda.get_device_name(i)}")
else:
    print("No GPU found.")


CUDA available: True
Number of GPUs: 1
GPU 0: Tesla T4


In [36]:
import os
import torch

def select_gpu():
    """
    Kiểm tra GPU khả dụng và tự động chọn GPU phù hợp.
    """
    if torch.cuda.is_available():
        num_gpus = torch.cuda.device_count()
        print(f"Number of GPUs available: {num_gpus}")

        # Duyệt qua các GPU khả dụng để tìm GPU ít sử dụng nhất
        available_gpus = [torch.cuda.get_device_name(i) for i in range(num_gpus)]
        print("Available GPUs:", available_gpus)

        for i in range(num_gpus):
            try:
                # Đặt GPU
                os.environ["CUDA_VISIBLE_DEVICES"] = str(i)
                device = torch.device(f"cuda:{i}")
                torch.cuda.set_device(device)
                print(f"Using GPU: {torch.cuda.get_device_name(device.index)}")
                return device
            except Exception as e:
                print(f"GPU {i} is not suitable: {e}")

        print("No suitable GPU found. Falling back to CPU.")
        return torch.device("cpu")
    else:
        print("No GPUs available. Using CPU.")
        return torch.device("cpu")

# Tự động chọn GPU hoặc CPU
device = select_gpu()

# Kiểm tra lại thiết bị đang sử dụng
print(f"Final selected device: {device}")


Number of GPUs available: 1
Available GPUs: ['Tesla T4']
Using GPU: Tesla T4
Final selected device: cuda:0


In [37]:

class BERTIntentClassification(nn.Module):


    def __init__(self, model_name="bert-base-uncased", num_classes=10, dropout_rate=0.1, cache_dir = "huggingface"):
        super(BERTIntentClassification, self).__init__()
        self.bert = AutoModel.from_pretrained(model_name, cache_dir = cache_dir)
        # Get BERT hidden size
        hidden_size = self.bert.config.hidden_size
        self.ffnn = nn.Sequential(
            nn.Linear(hidden_size, hidden_size),
            nn.LayerNorm(hidden_size),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            nn.Linear(hidden_size, num_classes)
        )


    def freeze_bert(self):
        for param in self.bert.parameters():
            param.requires_grad = False


    def get_pooling(self, hidden_state, attention_mask):
        """
        Get mean pooled representation from BERT hidden states
        Args:
            hidden_state: BERT output containing hidden states
        Returns:
            pooled_output: Mean pooled representation of the sequence
        """
        # Get last hidden state
        last_hidden_state = hidden_state.last_hidden_state  # Shape: [batch_size, seq_len, hidden_size]

        if attention_mask is not None:
            # Expand attention mask to match hidden state dimensions
            attention_mask = attention_mask.unsqueeze(-1)  # [batch_size, seq_len, 1]

            # Mask out padding tokens
            masked_hidden = last_hidden_state * attention_mask

            # Calculate mean (sum / number of actual tokens)
            sum_hidden = torch.sum(masked_hidden, dim=1)  # [batch_size, hidden_size]
            count_tokens = torch.sum(attention_mask, dim=1)  # [batch_size, 1]
            pooled_output = sum_hidden / count_tokens
        else:
            # If no attention mask, simply take mean of all tokens
            pooled_output = torch.mean(last_hidden_state, dim=1)

        return pooled_output


    def forward(self, input_ids, attention_mask, **kwargs):
        """
        Forward pass of the model
        Args:
            input_ids: Input token IDs
            attention_mask: Attention mask for padding
        Returns:
            logits: Raw logits for each class
        """
        # Get BERT hidden states
        hidden_state = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask,
        )

        # Get pooled representation
        hidden_state_pooling = self.get_pooling(hidden_state=hidden_state, attention_mask=attention_mask)

        # Pass through FFNN classifier
        logits = self.ffnn(hidden_state_pooling)

        return logits


In [38]:
class TrainerCustom(Trainer):

    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        """
        How the loss is computed by Trainer. By default, all models return the loss in the first element.

        Subclass and override for custom behavior.
        """
        if "labels" in inputs:
            labels = inputs.pop("labels")
        else:
            labels = None

        # Sử dụng nn.CrossEntropyLoss() thay vì nn.CrossEntropy
        cross_entropy_loss = nn.CrossEntropyLoss()

        # Chạy mô hình và nhận đầu ra (logits)
        outputs = model(**inputs)

        # Đảm bảo lấy logits từ outputs (mô hình trả về tuple, lấy phần tử đầu tiên là logits)
        logits = outputs

        # Tính toán loss
        loss = cross_entropy_loss(logits, labels)

        # Trả về loss và outputs nếu cần
        return (loss, outputs) if return_outputs else loss


# 1. Load Dataset and with Dataloader


**Cách kết hợp question và answer:**

1. **Ghép nối trực tiếp:** Bạn có thể kết hợp câu hỏi và câu trả lời thành một chuỗi duy nhất, sử dụng một ký tự đặc biệt hoặc dấu phân cách để tách biệt chúng. Ví dụ:

   ```python
   combined_text = question + " [SEP] " + answer
   ```

   Trong đó, `[SEP]` là một token đặc biệt thường được sử dụng trong các mô hình như BERT để phân tách các đoạn văn bản khác nhau.

2. **Sử dụng token đặc biệt:** Một số mô hình hỗ trợ các token đặc biệt để đánh dấu bắt đầu và kết thúc của câu hỏi và câu trả lời. Ví dụ:

   ```python
   combined_text = "[CLS] " + question + " [SEP] " + answer + " [SEP]"
   ```

   - `[CLS]`: Token đánh dấu bắt đầu chuỗi (thường dùng trong BERT).
   - `[SEP]`: Token phân tách giữa các phần của chuỗi.



Ví dụ thực tế về chuỗi chuẩn:

Một câu/đoạn duy nhất:
```
[CLS] This is the first sentence. [SEP]
```
Hai câu/đoạn (ví dụ: câu hỏi và trả lời):
```
[CLS] What is your name? [SEP] My name is John. [SEP]
```
Nhiều câu/đoạn (3 đoạn):
```
[CLS] Question 1 [SEP] Answer 1 [SEP] Extra information [SEP]
```

```
                          input_ids  intent
0  [CLS] Cậu có muốn tiếp tục không? [SEP]  silence
1                          [CLS] [SEP]  silence

```

In [39]:
import pandas as pd
from datasets import Dataset

def combine_with_special_tokens(row, text_columns, cls_token="[CLS]", sep_token="[SEP]"):
    """
    Thêm các token đặc biệt vào chuỗi kết hợp từ các cột văn bản.

    Args:
        row (pd.Series): Dòng dữ liệu từ DataFrame.
        text_columns (list): Danh sách các cột văn bản cần kết hợp.
        cls_token (str): Token bắt đầu câu.
        sep_token (str): Token phân cách.

    Returns:
        str: Chuỗi văn bản đã thêm token đặc biệt.
    """
    tokens = [cls_token]  # Thêm [CLS] đầu tiên

    # Thêm nội dung từ các cột văn bản
    for col in text_columns:
        if pd.notna(row[col]) and row[col].strip():  # Kiểm tra không rỗng
            tokens.append(row[col].strip())
            tokens.append(sep_token)  # Thêm [SEP] sau mỗi đoạn

    # Nếu không có nội dung nào được thêm, chỉ giữ lại [CLS] và [SEP]
    if len(tokens) == 1:
        tokens.append(sep_token)

    return " ".join(tokens)

def load_xlsx_dataset(xlsx_path, text_columns, label_column, cls_token="[CLS]", sep_token="[SEP]"):
    """
    Tải dataset từ file Excel (.xlsx) và xử lý dữ liệu.

    Args:
        xlsx_path (str): Đường dẫn đến file .xlsx.
        text_columns (list): Danh sách các cột cần ghép để tạo văn bản đầu vào.
        label_column (str): Tên cột chứa nhãn.
        cls_token (str): Token bắt đầu câu.
        sep_token (str): Token phân cách.

    Returns:
        Dataset: Tập dữ liệu đã xử lý.
    """
    # Đọc file Excel bằng pandas
    df = pd.read_excel(xlsx_path)

    # Kiểm tra các cột cần thiết
    for col in text_columns + [label_column]:
        if col not in df.columns:
            raise ValueError(f"Missing required column: {col}")

    # Ghép các cột text lại thành một chuỗi duy nhất với token đặc biệt
    df["input_ids"] = df.apply(lambda row: combine_with_special_tokens(row, text_columns, cls_token, sep_token), axis=1)

    # Đổi tên cột nhãn
    df = df.rename(columns={label_column: "label"})

    # Chuyển đổi DataFrame thành Dataset
    dataset = Dataset.from_pandas(df[["input_ids", "label"]])

    return dataset

# Sử dụng hàm
xlsx_path = "/content/processed_data_example_v2.xlsx"  # Đường dẫn file Excel
text_columns = ["question", "answer"]  # Các cột cần ghép
label_column = "intent"  # Cột chứa nhãn

# Tải dataset từ Excel
dataset = load_xlsx_dataset(xlsx_path, text_columns, label_column)

# Kiểm tra dữ liệu
print(dataset)

# Lấy 10 mẫu đầu tiên
sample_dataset = dataset.select(range(20))
print(sample_dataset)

# In thử một hàng
print("First row in dataset:")
print(sample_dataset[0])
print(sample_dataset[11])


Dataset({
    features: ['input_ids', 'label'],
    num_rows: 120
})
Dataset({
    features: ['input_ids', 'label'],
    num_rows: 20
})
First row in dataset:
{'input_ids': "[CLS] Cậu có thể kể tên một số hành động bắt đầu bằng từ 'play' không? [SEP] Tớ có thể nói 'play football' và 'play basketball'. [SEP]", 'label': 'intent_positive'}
{'input_ids': '[CLS] Cậu có muốn tiếp tục không? [SEP]', 'label': 'silence'}


In [40]:
def check_invalid_samples(dataset):
    invalid_samples = []
    for idx, sample in enumerate(dataset):
        if not isinstance(sample["input_ids"], str) or sample["input_ids"].strip() == "":
            invalid_samples.append((idx, sample))
    return invalid_samples

# Kiểm tra dữ liệu không hợp lệ
invalid_samples = check_invalid_samples(dataset)
print("\n===== Invalid Samples =====")
print(invalid_samples)



===== Invalid Samples =====
[]


In [41]:
# Tự động phát hiện nhãn và tạo ánh xạ nhãn
def create_label_mapping(dataset_list):
    """
    Tự động phát hiện tất cả các nhãn từ danh sách dataset và ánh xạ chúng thành số nguyên.
    """
    all_labels = set()
    for dataset in dataset_list:
        all_labels.update(dataset["label"])  # Tập hợp tất cả các nhãn từ dataset

    label_to_int = {label: idx for idx, label in enumerate(sorted(all_labels))}
    print(f"Ánh xạ nhãn: {label_to_int}")
    return label_to_int

# Hàm chuyển đổi nhãn
def preprocess_labels(example, label_to_int):
    example["label"] = label_to_int.get(example["label"], -1)  # Gán -1 cho nhãn không hợp lệ
    return example

# Tạo ánh xạ nhãn
label_mapping = create_label_mapping([dataset])

# Áp dụng chuyển đổi nhãn
dataset = dataset.map(lambda example: preprocess_labels(example, label_mapping))

# Kiểm tra kết quả
print(dataset)

# Truy cập mẫu cụ thể
sample_dataset = dataset.select(range(10))  # Lấy 10 mẫu đầu tiên
print(sample_dataset)

# In thử 1 hàng trong sample_dataset
print("First row in sample_dataset:")
print(sample_dataset[1])

Ánh xạ nhãn: {'intent_fallback': 0, 'intent_learn_more': 1, 'intent_negative': 2, 'intent_neutral': 3, 'intent_positive': 4, 'silence': 5}


Map:   0%|          | 0/120 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids', 'label'],
    num_rows: 120
})
Dataset({
    features: ['input_ids', 'label'],
    num_rows: 10
})
First row in sample_dataset:
{'input_ids': "[CLS] Cậu có biết thêm từ nào khác không? [SEP] Tớ biết 'play games' nữa. [SEP]", 'label': 4}


In [53]:
def split_dataset(dataset, test_size=0.2, seed=42):
    """
    Chia dataset thành tập train và test.

    Args:
        dataset (Dataset): Tập dữ liệu đầy đủ.
        test_size (float): Tỷ lệ dữ liệu test (0.0 - 1.0).
        seed (int): Seed để chia dữ liệu ngẫu nhiên.

    Returns:
        tuple: (train_dataset, test_dataset) - Tập train và test.
    """
    if not (0.0 < test_size < 1.0):
        raise ValueError("test_size phải nằm trong khoảng (0.0, 1.0)")
    if len(dataset) < 2:
        raise ValueError("Dataset phải có ít nhất 2 mẫu để chia.")

    train_test_split = dataset.train_test_split(test_size=test_size, seed=seed)
    print(f"Chia dataset: {len(train_test_split['train'])} mẫu train, {len(train_test_split['test'])} mẫu test")
    return train_test_split["train"], train_test_split["test"]

# Chia dataset
train_dataset, test_dataset = split_dataset(dataset, test_size=0.3)

# Kiểm tra dữ liệu
print("Train dataset:", train_dataset)
print("Test dataset:", test_dataset)

# Truy cập mẫu cụ thể
sample_train_dataset = train_dataset.select(range(8))  # Lấy 10 mẫu đầu tiên từ train
sample_test_dataset = test_dataset.select(range(5))    # Lấy 10 mẫu đầu tiên từ test

print("Sample train dataset:", sample_train_dataset)
print("Sample test dataset:", sample_test_dataset)

Chia dataset: 84 mẫu train, 36 mẫu test
Train dataset: Dataset({
    features: ['input_ids', 'label'],
    num_rows: 84
})
Test dataset: Dataset({
    features: ['input_ids', 'label'],
    num_rows: 36
})
Sample train dataset: Dataset({
    features: ['input_ids', 'label'],
    num_rows: 8
})
Sample test dataset: Dataset({
    features: ['input_ids', 'label'],
    num_rows: 5
})


# 2. Tokenizer

In [43]:
# Calculate the number of unique labels
print(label_mapping)
number_label = len(label_mapping)
print("Number of unique labels:", number_label)

{'intent_fallback': 0, 'intent_learn_more': 1, 'intent_negative': 2, 'intent_neutral': 3, 'intent_positive': 4, 'silence': 5}
Number of unique labels: 6


In [44]:





# Bước 2: Chuẩn bị tokenizer và token hóa dữ liệu
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name, cache_dir = "huggingface")
model = BERTIntentClassification(
    model_name=model_name,
    num_classes=6
)
model.freeze_bert() # Froze Layer BERT
max_seq_length = 512


def collate_fn(features):
    inputs = []
    labels = []
    for element in features:
        inputs.append(element.get("input_ids"))
        labels.append(element.get("label"))

    labels = torch.tensor(labels, dtype=torch.long)

    token_inputs = tokenizer(
        inputs,
        add_special_tokens=True,
        truncation=True,
        padding=True,
        max_length=max_seq_length,
        return_overflowing_tokens=False,
        return_length=False,
        return_tensors="pt",
    )
    token_inputs.update({
        "labels": labels,
    })
    return token_inputs


# 3. Train Model

## 3.1 Log Wandb

In [45]:
!pip install --upgrade wandb

In [46]:
!pip install python-dotenv

In [47]:
from dotenv import load_dotenv
import os

# Load biến môi trường từ file .env
load_dotenv()

# Lấy key từ biến môi trường
wandb_api_key = os.getenv("WANDB_API_KEY")
print(wandb_api_key[:5])

c8767


In [48]:
import wandb
import os

# Lấy API key từ biến môi trường và đăng nhập
wandb.login(key=os.getenv("WANDB_API_KEY"))


wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


True

Cách thiết lập thông qua TrainingArguments
Khi sử dụng Trainer, bạn có thể đặt tên dự án trực tiếp trong TrainingArguments bằng cách sử dụng tham số report_to và run_name. Tuy nhiên, để đặt project, bạn cần khởi tạo một phiên wandb trước hoặc truyền cấu hình này thông qua wandb.init().

Điều chỉnh TrainingArguments:
```python
training_args = TrainingArguments(
    output_dir="./results_",          # Thư mục lưu kết quả
    eval_strategy="epoch",           # Đánh giá sau mỗi epoch
    learning_rate=2e-4,
    per_device_train_batch_size=128,
    per_device_eval_batch_size=128,
    num_train_epochs=5,
    weight_decay=0.01,
    logging_dir="./logs",            # Thư mục lưu log
    logging_strategy="steps",        # Log theo steps
    logging_steps=10,                # Log sau mỗi 10 bước
    save_strategy="epoch",           # Lưu checkpoint sau mỗi epoch
    save_total_limit=3,              # Lưu tối đa 3 checkpoint
    report_to="wandb",               # Báo cáo log tới wandb
    run_name="bert_run_1"            # Tên phiên chạy trên wandb
)
```

## 3.2 Train

In [49]:
# from concurrent.futures import ThreadPoolExecutor
# import wandb
# import os
# import shutil
# import time

# class TrainerCustom(Trainer):
#     def __init__(self, *args, save_every_n_epochs=10, **kwargs):
#         super().__init__(*args, **kwargs)
#         if torch.cuda.is_available():
#             print(f"Trainer is running on GPU: {torch.cuda.get_device_name(torch.cuda.current_device())}")
#         else:
#             print("Trainer is running on CPU.")

#         self.best_eval_loss = float("inf")  # Giá trị loss tốt nhất ban đầu
#         self.save_every_n_epochs = save_every_n_epochs  # Tần suất lưu lên WandB
#         self.best_model_info = {"epoch": None, "loss": None}
#         self.last_saved_epoch = 0  # Epoch cuối cùng đã lưu Best Model và Last Model
#         self.executor = ThreadPoolExecutor(max_workers=3)  # Cho phép tối đa 2 luồng song song

#     def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
#         """
#         How the loss is computed by Trainer. By default, all models return the loss in the first element.

#         Subclass and override for custom behavior.
#         """

#         # # Kiểm tra thiết bị của mô hình và dữ liệu
#         # print("Model device:", next(model.parameters()).device)
#         # print("Input device:", inputs["input_ids"].device)
#         if "labels" in inputs:
#             labels = inputs.pop("labels")
#         else:
#             labels = None

#         # Sử dụng nn.CrossEntropyLoss() thay vì nn.CrossEntropy
#         cross_entropy_loss = nn.CrossEntropyLoss()

#         # Chạy mô hình và nhận đầu ra (logits)
#         outputs = model(**inputs)

#         # Đảm bảo lấy logits từ outputs (mô hình trả về tuple, lấy phần tử đầu tiên là logits)
#         logits = outputs

#         if labels is None:
#             print("Labels are None during compute_loss.")
#         if logits is None:
#             print("Logits are None during compute_loss.")

#         # Tính toán loss
#         loss = cross_entropy_loss(logits, labels)

#         # Trả về loss và outputs nếu cần
#         return (loss, outputs) if return_outputs else loss

#     def async_save_model(self, model_dir, artifact_name, metadata=None):
#         """
#         Lưu mô hình vào local và đồng bộ lên WandB trong luồng song song.
#         """
#         def save():
#             start_time = time.time()
#             try:
#                 # Xóa tất cả các thư mục tmp_best_model_ trước đó
#                 for folder in os.listdir("."):
#                     if folder.startswith("tmp_best_model_epoch_") and folder != model_dir:
#                         shutil.rmtree(folder, ignore_errors=True)
#                         print(f"Removed old temporary directory: {folder}")

#                 # Lưu mô hình vào thư mục tạm
#                 self.save_model(model_dir)

#                 # Đồng bộ lên WandB
#                 artifact = wandb.Artifact(artifact_name, type="model")
#                 artifact.add_dir(model_dir)
#                 if metadata:
#                     artifact.metadata = metadata
#                 wandb.log_artifact(artifact)
#             except Exception as e:
#                 print(f"Error during saving or syncing model {artifact_name}: {e}")
#             finally:
#                 # Xóa thư mục tạm hiện tại sau khi đồng bộ
#                 try:
#                     shutil.rmtree(model_dir, ignore_errors=True)
#                     print(f"Successfully removed temporary directory: {model_dir}")
#                 except Exception as e:
#                     print(f"Error removing temporary directory {model_dir}: {e}")

#             elapsed_time = time.time() - start_time
#             print(f"Model saved and uploaded to WandB: {artifact_name} in {elapsed_time:.2f} seconds")

#         self.executor.submit(save)




#     def evaluate(self, eval_dataset=None, ignore_keys=None, metric_key_prefix: str = "eval"):
#         metrics = super().evaluate(eval_dataset, ignore_keys, metric_key_prefix)
#         eval_loss = metrics.get("eval_loss")

#         # Cập nhật Best Model nếu eval_loss giảm
#         # Lưu Best Model ngay khi eval_loss giảm (local).
#         # Chỉ đồng bộ lên WandB mỗi 10 epochs.

#         if eval_loss is not None and eval_loss < self.best_eval_loss:
#             print(f"New best eval_loss: {eval_loss}")
#             self.best_eval_loss = eval_loss
#             self.best_model_info = {"epoch": self.state.epoch, "loss": eval_loss}

#             # Log thông tin Best Model lên WandB
#             wandb.log({
#                 "best_eval_loss": self.best_eval_loss,
#                 "best_model_epoch": self.best_model_info.get("epoch", -1)
#             })

#             # Lưu Best Model vào thư mục tạm (local)
#             best_model_dir = f"./tmp_best_model_epoch_{int(self.state.epoch)}"
#             self.save_model(best_model_dir)

#             # Đồng bộ lên WandB mỗi 10 epochs
#             if int(self.state.epoch) % self.save_every_n_epochs == 0:
#                 artifact_name = f"best_model_epoch_{int(self.state.epoch)}"
#                 self.async_save_model(best_model_dir, artifact_name, self.best_model_info)

#         return metrics

#     def save_last_model(self):
#         """
#         Lưu Last Model lên WandB sau mỗi N epochs.
#         """
#         if int(self.state.epoch) % self.save_every_n_epochs == 0 and int(self.state.epoch) != self.last_saved_epoch:
#             print(f"Saving Last Model at epoch {self.state.epoch} to WandB...")
#             last_model_dir = f"./tmp_last_model_epoch_{int(self.state.epoch)}"
#             artifact_name = f"last_model_epoch_{int(self.state.epoch)}"
#             self.async_save_model(last_model_dir, artifact_name)

#             # Log thông tin Last Model lên WandB
#             wandb.log({
#                 "last_model_epoch": self.state.epoch
#             })

#             # Cập nhật epoch cuối cùng đã lưu
#             self.last_saved_epoch = int(self.state.epoch)

#     def train(self, *args, **kwargs):
#         result = super().train(*args, **kwargs)

#         # Sau mỗi epoch, lưu Last Model lên WandB
#         self.save_last_model()
#         # Chờ tất cả các luồng lưu hoàn thành trước khi kết thúc
#         self.executor.shutdown(wait=True)

#         return result


# # Bước 6: Cài đặt tham số huấn luyện
# training_args = TrainingArguments(
#     output_dir="./result__s",          # Thư mục lưu kết quả
#     eval_strategy="epoch",    # Đánh giá sau mỗi epoch
#     learning_rate=2e-4,
#     per_device_train_batch_size=128,
#     per_device_eval_batch_size=128,
#     num_train_epochs=30,
#     weight_decay=0.01,
#     logging_dir="./logs",
#     logging_strategy="steps",
#     logging_steps=1,  # Ghi logs mỗi 500 bước huấn luyện
#     save_strategy="no",          # Lưu trọng số sau mỗi epoch
#     save_total_limit=3,
#     label_names = ["labels"],
#     report_to="wandb",
#     run_name="bert_run_3"
# )


# import wandb

# # Khởi tạo wandb
# wandb.init(
#     project="bert-intent-classification",  # Tên dự án
#     name="bert_run_3",                     # Tên phiên chạy
#     config={"gpu": torch.cuda.get_device_name(torch.cuda.current_device()) if torch.cuda.is_available() else "CPU"}
# )

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model = model.to(device)
# if torch.cuda.is_available():
#     print(f"Trainer is running on GPU: {torch.cuda.get_device_name(torch.cuda.current_device())}")
# else:
#     print("Trainer is running on CPU.")

# trainer = TrainerCustom(
#     model=model,
#     args=training_args,
#     train_dataset=sample_train_dataset,
#     eval_dataset=sample_test_dataset,
#     data_collator=collate_fn,
#     save_every_n_epochs=10  # Lưu Best Model và Last Model mỗi 10 epochs
# )

# trainer.train()


# wandb.finish()


## 3.3 Continue Train

### Ver 1.2.3

Thui, ko lưu local nữa, lưu tất trên wandb đi.
- Với best model: lưu lên wandb khi loss giảm và đã sau 10 epochs  
(Lưu Best Model ngay khi eval_loss giảm ở local, sau 10 epochs thì đồng bộ cái best lên wandb, sau đó xoá các file best ở local).
Chỉ đồng bộ lên WandB mỗi 10 epochs.)
- Với last model: lưu lên wandb sau mỗi 10 epochs. (lưu local trước -> đồng bộ lên wandb sẽ xoá file local)
+, Trong quá trình lưu thì việc training vẫn diễn ra Parallel

đều lưu đầy đủ toàn bộ tham số để có thể train thêm từ cả ở best model và last model

In [55]:
from concurrent.futures import ThreadPoolExecutor
import wandb
import os
import shutil
import time

class TrainerCustom(Trainer):
    def __init__(self, *args, save_every_n_epochs=10, **kwargs):
        super().__init__(*args, **kwargs)
        if torch.cuda.is_available():
            print(f"Trainer is running on GPU: {torch.cuda.get_device_name(torch.cuda.current_device())}")
        else:
            print("Trainer is running on CPU.")

        self.best_eval_loss = float("inf")  # Giá trị loss tốt nhất ban đầu
        self.save_every_n_epochs = save_every_n_epochs  # Tần suất lưu lên WandB
        self.best_model_info = {"epoch": None, "loss": None}
        self.last_saved_epoch = 0  # Epoch cuối cùng đã lưu Best Model và Last Model
        self.executor = ThreadPoolExecutor(max_workers=3)  # Cho phép tối đa 2 luồng song song

    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        """
        How the loss is computed by Trainer. By default, all models return the loss in the first element.

        Subclass and override for custom behavior.
        """

        # # Kiểm tra thiết bị của mô hình và dữ liệu
        # print("Model device:", next(model.parameters()).device)
        # print("Input device:", inputs["input_ids"].device)
        if "labels" in inputs:
            labels = inputs.pop("labels")
        else:
            labels = None

        # Sử dụng nn.CrossEntropyLoss() thay vì nn.CrossEntropy
        cross_entropy_loss = nn.CrossEntropyLoss()

        # Chạy mô hình và nhận đầu ra (logits)
        outputs = model(**inputs)

        # Đảm bảo lấy logits từ outputs (mô hình trả về tuple, lấy phần tử đầu tiên là logits)
        logits = outputs

        if labels is None:
            print("Labels are None during compute_loss.")
        if logits is None:
            print("Logits are None during compute_loss.")

        # Tính toán loss
        loss = cross_entropy_loss(logits, labels)

        # Trả về loss và outputs nếu cần
        return (loss, outputs) if return_outputs else loss

    def async_save_model(self, model_dir, artifact_name, metadata=None):
        """
        Lưu mô hình vào local và đồng bộ lên WandB trong luồng song song.
        """
        def save():
            start_time = time.time()
            try:
                # Xóa tất cả các thư mục tmp_best_model_ trước đó
                for folder in os.listdir("."):
                    if folder.startswith("tmp_best_model_epoch_") and folder != model_dir:
                        shutil.rmtree(folder, ignore_errors=True)
                        print(f"Removed old temporary directory: {folder}")

                # Lưu mô hình vào thư mục tạm
                self.save_model(model_dir)

                # Đồng bộ lên WandB
                artifact = wandb.Artifact(artifact_name, type="model")
                artifact.add_dir(model_dir)
                if metadata:
                    artifact.metadata = metadata
                wandb.log_artifact(artifact)
            except Exception as e:
                print(f"Error during saving or syncing model {artifact_name}: {e}")
            finally:
                # Xóa thư mục tạm hiện tại sau khi đồng bộ
                try:
                    shutil.rmtree(model_dir, ignore_errors=True)
                    print(f"Successfully removed temporary directory: {model_dir}")
                except Exception as e:
                    print(f"Error removing temporary directory {model_dir}: {e}")

            elapsed_time = time.time() - start_time
            print(f"Model saved and uploaded to WandB: {artifact_name} in {elapsed_time:.2f} seconds")

        self.executor.submit(save)




    def evaluate(self, eval_dataset=None, ignore_keys=None, metric_key_prefix: str = "eval"):
        metrics = super().evaluate(eval_dataset, ignore_keys, metric_key_prefix)
        eval_loss = metrics.get("eval_loss")

        # Cập nhật Best Model nếu eval_loss giảm
        # Lưu Best Model ngay khi eval_loss giảm (local).
        # Chỉ đồng bộ lên WandB mỗi 10 epochs.

        if eval_loss is not None and eval_loss < self.best_eval_loss:
            print(f"New best eval_loss: {eval_loss}")
            self.best_eval_loss = eval_loss
            self.best_model_info = {"epoch": self.state.epoch, "loss": eval_loss}

            # Log thông tin Best Model lên WandB
            wandb.log({
                "best_eval_loss": self.best_eval_loss,
                "best_model_epoch": self.best_model_info.get("epoch", -1)
            })

            # Lưu Best Model vào thư mục tạm (local)
            best_model_dir = f"./tmp_best_model_epoch_{int(self.state.epoch)}"
            self.save_model(best_model_dir)

            # Đồng bộ lên WandB mỗi 10 epochs
            if int(self.state.epoch) % self.save_every_n_epochs == 0:
                artifact_name = f"best_model_epoch_{int(self.state.epoch)}"
                self.async_save_model(best_model_dir, artifact_name, self.best_model_info)

        return metrics

    def save_last_model(self):
        """
        Lưu Last Model lên WandB sau mỗi N epochs.
        """
        if int(self.state.epoch) % self.save_every_n_epochs == 0 and int(self.state.epoch) != self.last_saved_epoch:
            print(f"Saving Last Model at epoch {self.state.epoch} to WandB...")
            last_model_dir = f"./tmp_last_model_epoch_{int(self.state.epoch)}"
            artifact_name = f"last_model_epoch_{int(self.state.epoch)}"
            self.async_save_model(last_model_dir, artifact_name)

            # Log thông tin Last Model lên WandB
            wandb.log({
                "last_model_epoch": self.state.epoch
            })

            # Cập nhật epoch cuối cùng đã lưu
            self.last_saved_epoch = int(self.state.epoch)

    def train(self, *args, **kwargs):
        result = super().train(*args, **kwargs)

        # Sau mỗi epoch, lưu Last Model lên WandB
        self.save_last_model()
        # Chờ tất cả các luồng lưu hoàn thành trước khi kết thúc
        self.executor.shutdown(wait=True)

        return result





- model.safetensors là tệp trọng số được lưu bằng thư viện safetensors, không phải định dạng PyTorch tiêu chuẩn (.bin hoặc .pt).
- Để tải tệp này, bạn cần sử dụng thư viện safetensors thay vì torch.load().

```
model = model.to(device)
model.load_state_dict(torch.load(weights_path, map_location="cpu"))
```

```
# Đường dẫn đến file trọng số
weights_path = os.path.join(artifact_dir, "model.safetensors")

from safetensors.torch import load_file

weights = load_file(weights_path)  # Tải trọng số từ tệp .safetensors

# Áp dụng trọng số vào mô hình
model.load_state_dict(weights)
model = model.to(device)
```

In [51]:
!pip install safetensors

In [58]:
import os
import json
import torch
from transformers import (
    AutoConfig,
    AutoTokenizer,
    TrainingArguments,
    Trainer,
)
from safetensors.torch import load_file
import wandb


# 1. Initialize WandB and download artifact
run = wandb.init(project="bert-intent-classification", name="continue_training")
artifact = run.use_artifact('doanngoccuong_nh/bert-intent-classification/last_model_epoch_30:v3', type='model')
artifact_dir = artifact.download()
print("Files in artifact_dir:", os.listdir(artifact_dir))

# 2. Create config.json if not available
config_path = os.path.join(artifact_dir, "config.json")
if not os.path.exists(config_path):
    print("Creating config.json...")
    config = AutoConfig.from_pretrained("bert-base-uncased")
    with open(config_path, "w") as f:
        json.dump(config.to_dict(), f, indent=4)
    print(f"Config.json created at {config_path}")
else:
    print("Config.json already exists.")

# 3. Load the model
print("Loading model weights...")
weights_path = os.path.join(artifact_dir, "model.safetensors")
model = BERTIntentClassification(model_name="bert-base-uncased", num_classes=6)  # Ensure num_classes matches your dataset
weights = load_file(weights_path)
model.load_state_dict(weights)
print("Model loaded successfully.")

# Move model to device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
if torch.cuda.is_available():
    print(f"Trainer is running on GPU: {torch.cuda.get_device_name(torch.cuda.current_device())}")
else:
    print("Trainer is running on CPU.")

# 4. Load or create tokenizer
print("Loading tokenizer...")
tokenizer_path = os.path.join(artifact_dir, "tokenizer_config.json")
if os.path.exists(tokenizer_path):
    tokenizer = AutoTokenizer.from_pretrained(artifact_dir)
else:
    print("Tokenizer not found. Loading from bert-base-uncased...")
    tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
    tokenizer.save_pretrained(artifact_dir)
    print(f"Tokenizer saved to {artifact_dir}.")

# 5. Load training arguments and trainer state
trainer_state_path = os.path.join(artifact_dir, "trainer_state.json")
if os.path.exists(trainer_state_path):
    print(f"Loading trainer state from {trainer_state_path}...")
else:
    print("Trainer state not found. Training will start fresh.")

# 6. Define training arguments
training_args = TrainingArguments(
    output_dir="./result__s",
    eval_strategy="epoch",
    learning_rate=2e-4,
    per_device_train_batch_size=128,
    per_device_eval_batch_size=128,
    num_train_epochs=30,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_strategy="steps",
    logging_steps=10,
    save_strategy="epoch",
    save_total_limit=3,
    label_names=["labels"],
    report_to="wandb",
    run_name="bert_continue_training"
)


# 8. Initialize Trainer
print("Initializing Trainer...")
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    data_collator=collate_fn,
    # save_every_n_epochs=10  # Lưu Best Model và Last Model mỗi 10 epochs
)

# 9. Continue Training
print("Starting training...")
trainer.train()

# 10. Finish WandB run
wandb.finish()


wandb: Downloading large artifact last_model_epoch_30:v3, 419.96MB. 3 files... 
wandb:   3 of 3 files downloaded.  
Done. 0:0:1.1


Files in artifact_dir: ['models--bert-base-uncased', 'model.safetensors', 'trainer_state.json', 'vocab.txt', 'tokenizer_config.json', 'special_tokens_map.json', 'training_args.bin', 'tokenizer.json', '.locks', 'config.json']
Config.json already exists.
Loading model weights...
Model loaded successfully.
Trainer is running on GPU: Tesla T4
Loading tokenizer...
Loading trainer state from /content/artifacts/last_model_epoch_30:v3/trainer_state.json...
Initializing Trainer...
Starting training...


RuntimeError: grad can be implicitly created only for scalar outputs

Đúng vậy, trong đoạn mã bạn cung cấp, `trainer.evaluate()` được thực hiện một cách tự động bởi lớp `Trainer` trong thư viện `transformers`. Cụ thể:

### Trong TrainingArguments:
```python
training_args = TrainingArguments(
    ...
    eval_strategy="epoch",  # Đánh giá sau mỗi epoch
    ...
)
```
**`eval_strategy="epoch"`** có nghĩa là quá trình đánh giá (evaluation) sẽ tự động được thực hiện sau mỗi epoch, sử dụng `eval_dataset` mà bạn đã cung cấp trong `TrainerCustom`.

### Trong `TrainerCustom`:
Trong lớp `TrainerCustom`, phương thức `evaluate()` đã được override. Bên trong, nó:
1. Gọi phương thức `super().evaluate()` từ lớp cha `Trainer`, thực hiện việc tính toán loss và các metric.
2. Lưu thông tin về Best Model nếu phát hiện `eval_loss` giảm so với trước đó.
3. Ghi log kết quả lên WandB.

Vì vậy, trong khi huấn luyện (`trainer.train()`), `trainer.evaluate()` được gọi tự động sau mỗi epoch để thực hiện đánh giá và lưu Best Model.

---

### Kết luận:
Bạn không cần gọi riêng `trainer.evaluate()` trong lúc training nếu đã cấu hình `eval_strategy="epoch"`. Tuy nhiên, nếu bạn muốn đánh giá mô hình ở một thời điểm cụ thể ngoài quá trình training (ví dụ, sau khi huấn luyện xong), bạn vẫn có thể gọi `trainer.evaluate()` thủ công.

In [ ]:
# # Bước 9: Đánh giá trên tập kiểm tra
# trainer.evaluate()

# Inference

Dưới đây là bảng chi tiết hơn so sánh giữa `safetensors` và `pytorch_model.bin` dựa trên các tiêu chí quan trọng:

| **Tiêu chí**                        | **Safetensors**                                                                                       | **Pytorch_model.bin**                                                                                     |
|-------------------------------------|-------------------------------------------------------------------------------------------------------|-----------------------------------------------------------------------------------------------------------|
| **Định dạng**                       | Lưu trữ tensor (trọng số) với định dạng nhị phân an toàn, không lưu metadata.                         | Lưu cả tensor và metadata, hỗ trợ trạng thái optimizer, scheduler, và nhiều thông tin khác.              |
| **Kích thước tệp**                  | Nhẹ hơn, tối ưu hóa kích thước bằng cách loại bỏ metadata không cần thiết.                             | Lớn hơn, do lưu trữ đầy đủ thông tin của mô hình, bao gồm metadata.                                       |
| **Tốc độ tải (Load speed)**         | **Nhanh hơn** nhờ tải tensor trực tiếp từ ổ đĩa.                                                      | Chậm hơn, cần tải toàn bộ tệp vào RAM trước khi sử dụng.                                                  |
| **Hiệu quả bộ nhớ (Memory efficiency)** | Hỗ trợ **lazy loading**, chỉ tải các tensor cần thiết, tiết kiệm RAM.                                   | Không hỗ trợ lazy loading, cần bộ nhớ lớn để tải toàn bộ mô hình.                                         |
| **Bảo mật (Security)**              | **Rất an toàn**, không hỗ trợ thực thi mã nhị phân, giảm rủi ro bảo mật.                               | **Không an toàn** nếu tệp bị chỉnh sửa ác ý, có thể thực thi mã nhị phân qua pickle.                      |
| **Phổ biến (Popularity)**           | Ít phổ biến hơn, cần thư viện `safetensors` hoặc phiên bản Hugging Face >= 4.25.                       | Rất phổ biến, tiêu chuẩn trong cộng đồng PyTorch và Hugging Face.                                         |
| **Khả năng tương thích (Compatibility)** | Hỗ trợ các framework hiện đại (PyTorch, TensorFlow, JAX).                                              | Tương thích mạnh mẽ với PyTorch, nhưng hạn chế khi chuyển đổi giữa các framework khác.                    |
| **Triển khai inference (Deployment)**| Phù hợp với môi trường hiện đại, cần cài thư viện bổ sung (`safetensors`).                             | Tương thích rộng rãi trên mọi thiết bị và môi trường, không cần cài thêm thư viện.                        |
| **Hỗ trợ thiết bị cũ**              | Yêu cầu môi trường hiện đại, không tương thích tốt với các thiết bị hoặc framework cũ.                 | Hỗ trợ tốt trên cả các thiết bị hoặc thư viện cũ.                                                         |
| **Lưu trạng thái mô hình (Model state)** | Chỉ lưu tensor (trọng số mô hình), không lưu trạng thái optimizer hoặc scheduler.                     | Lưu đầy đủ trạng thái, phù hợp cho việc fine-tuning hoặc khôi phục huấn luyện.                            |
| **Tốc độ triển khai inference**     | Nhanh hơn, đặc biệt khi tải mô hình lớn trên CPU.                                                     | Chậm hơn một chút, nhưng không đáng kể trên GPU hiện đại.                                                 |
| **Hỗ trợ khi huấn luyện (Training)**| Cần cấu hình thêm (`save_safetensors=True`) trong `TrainingArguments`.                                | Dễ dàng sử dụng mặc định, không cần cấu hình thêm.                                                        |
| **Hỗ trợ tối ưu hóa (Optimizer support)** | Không hỗ trợ lưu optimizer, cần xử lý riêng nếu tiếp tục huấn luyện.                                   | Hỗ trợ lưu optimizer, thuận tiện cho việc khôi phục và tiếp tục huấn luyện.                               |
| **Hỗ trợ trên WandB (Weights & Biases)** | Tích hợp tốt, giảm kích thước tệp khi đồng bộ mô hình.                                                 | Hỗ trợ đầy đủ, nhưng kích thước lớn hơn có thể ảnh hưởng đến tốc độ đồng bộ.                              |
| **Tính năng đặc biệt**              | - Hỗ trợ chia nhỏ mô hình lớn khi tải.<br>- Tăng tốc inference trên môi trường sản xuất (production).  | - Tích hợp sâu với các công cụ debug và khôi phục huấn luyện.                                             |

---

### **Gợi ý sử dụng**
#### **Khi nào chọn `safetensors`:**
- Khi bạn ưu tiên tốc độ và bảo mật.
- Khi làm việc trên các môi trường hiện đại, yêu cầu hiệu suất cao.
- Khi kích thước tệp nhỏ gọn là ưu tiên (ví dụ: triển khai trên thiết bị biên).

#### **Khi nào chọn `pytorch_model.bin`:**
- Khi bạn cần tương thích rộng rãi với các thiết bị hoặc thư viện.
- Khi bạn cần lưu cả trạng thái optimizer và scheduler để tiếp tục huấn luyện.
- Khi triển khai trên các hệ thống cũ hoặc không hỗ trợ `safetensors`.

---

Nếu bạn cần thêm thông tin cụ thể hoặc muốn thử nghiệm một trường hợp thực tế, hãy cho mình biết nhé! 🚀

```
ValueError: Unrecognized model in /content/artifacts/last_model_epoch_30:v0. Should have a `model_type` key in its config.json, or contain one of the following strings in its name: albert, align, altclip, audio
```
Lỗi trên xảy ra do thư mục chứa mô hình tải về từ WandB (artifact_dir) không có tệp config.json hoặc tokenizer_config.json, là các tệp bắt buộc để AutoTokenizer.from_pretrained() xác định loại mô hình và tokenizer.

Cần cả **`config.json`** và **`tokenizer_config.json`** nếu bạn sử dụng `AutoTokenizer.from_pretrained()` để tải tokenizer, vì mỗi tệp đóng vai trò riêng biệt trong việc định cấu hình mô hình và tokenizer.

---

### **1. Vai trò của các tệp**

#### **`config.json`**
- **Mô tả mô hình (model configuration):**
  - Cấu trúc và các siêu tham số của mô hình, như:
    - `model_type` (ví dụ: `bert`, `gpt2`).
    - `hidden_size`, `num_attention_heads`, `num_hidden_layers`.
  - Dùng để khởi tạo mô hình bằng `AutoModel.from_pretrained()`.

#### **`tokenizer_config.json`**
- **Cấu hình tokenizer:**
  - Quy định các tham số liên quan đến tokenizer, như:
    - `do_lower_case`: Có chuyển đổi chữ hoa thành chữ thường không.
    - `model_type`: Loại mô hình liên kết với tokenizer (ví dụ: `bert`).
    - Các thông tin bổ sung như `max_length`, `padding_side`, `special_tokens_map`.

- **Cần thiết cho `AutoTokenizer.from_pretrained()`**, để tải chính xác tokenizer và cấu hình các bước xử lý đầu vào.

---

### **2. Khi nào cần cả hai tệp?**

- **Cần cả hai tệp nếu:**
  - Bạn sử dụng `AutoTokenizer.from_pretrained()` để tải tokenizer và muốn đảm bảo cấu hình đầy đủ.
  - Mô hình cần xử lý đầu vào đặc biệt (ví dụ: với `do_lower_case=True` hoặc sử dụng các token đặc biệt).
  - Bạn muốn đồng bộ hóa cấu hình giữa mô hình (`config.json`) và tokenizer (`tokenizer_config.json`).

- **Chỉ cần `config.json` nếu:**
  - Bạn chỉ tải mô hình bằng `AutoModel.from_pretrained()` mà không sử dụng tokenizer.

---

### **3. Hậu quả nếu thiếu `tokenizer_config.json`**

Nếu thiếu **`tokenizer_config.json`**:
- `AutoTokenizer.from_pretrained()` sẽ không thể tự động thiết lập các tham số như `do_lower_case` hoặc các token đặc biệt (`[CLS]`, `[SEP]`).
- Bạn sẽ cần phải chỉ định thủ công các tham số khi khởi tạo tokenizer, ví dụ:

```python
tokenizer = AutoTokenizer.from_pretrained(
    artifact_dir,
    do_lower_case=True,  # Thiết lập thủ công
    max_length=512
)
```

```
artifact_dir/
│
├── config.json
├── tokenizer_config.json
├── vocab.txt  # hoặc tokenizer.json
├── model.safetensors
├── training_args.bin

```

In [ ]:
import os
import json
import torch
from transformers import AutoTokenizer, AutoModel
import wandb

# 1. Tải mô hình từ artifact trên WandB
run = wandb.init(project="bert-intent-classification")  # Tên dự án trong WandB
artifact = run.use_artifact('doanngoccuong_nh/bert-intent-classification/last_model_epoch_30:v0', type='model')
artifact_dir = artifact.download()
print(os.listdir(artifact_dir))

In [ ]:
# import os
# import json
# import torch
# from transformers import AutoTokenizer, AutoModel
# import wandb

# # 1. Tải mô hình từ artifact trên WandB
# run = wandb.init(project="bert-intent-classification")  # Tên dự án trong WandB
# artifact = run.use_artifact('doanngoccuong_nh/bert-intent-classification/last_model_epoch_30:v0', type='model')
# artifact_dir = artifact.download()
# print("Files in artifact_dir:", os.listdir(artifact_dir))

# # Đường dẫn tệp cấu hình
# config_path = os.path.join(artifact_dir, "config.json")
# tokenizer_config_path = os.path.join(artifact_dir, "tokenizer_config.json")

# # 2. Tạo file config.json
# config = {
#     "model_type": "bert",
#     "hidden_size": 768,
#     "num_attention_heads": 12,
#     "num_hidden_layers": 12,
#     "vocab_size": 30522
# }

# with open(config_path, "w") as f:
#     json.dump(config, f, indent=4)

# print(f"Config.json created at {config_path}")

# # 3. Tạo file tokenizer_config.json
# tokenizer_config = {
#     "model_type": "bert",
#     "do_lower_case": True,
#     "max_length": 512,
#     "padding_side": "right",
#     "special_tokens_map": {
#         "[CLS]": "[CLS]",
#         "[SEP]": "[SEP]",
#         "[PAD]": "[PAD]"
#     }
# }

# with open(tokenizer_config_path, "w") as f:
#     json.dump(tokenizer_config, f, indent=4)

# print(f"Tokenizer_config.json created at {tokenizer_config_path}")

# # 4. Tải mô hình đã lưu và tokenizer
# model_path = artifact_dir  # Đường dẫn đến mô hình đã tải
# tokenizer = AutoTokenizer.from_pretrained(model_path)
# model = AutoModel.from_pretrained(model_path)

# # Chuyển mô hình sang chế độ đánh giá
# model.eval()
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model = model.to(device)

# print(f"Model loaded and running on device: {device}")

# # 5. Xử lý đầu vào
# sentence = "What is the weather like today?"
# inputs = tokenizer(
#     sentence,
#     return_tensors="pt",
#     truncation=True,
#     padding=True,
#     max_length=512
# )

# # Chuyển đầu vào sang thiết bị phù hợp
# inputs = {key: value.to(device) for key, value in inputs.items()}

# # 6. Thực hiện dự đoán
# with torch.no_grad():
#     outputs = model(**inputs)  # Truyền đầu vào qua mô hình
#     logits = outputs[0]  # Lấy logits từ đầu ra của mô hình
#     predicted_class = torch.argmax(logits, dim=1).item()  # Lấy nhãn dự đoán

# # 7. Mapping nhãn dự đoán sang tên nhãn
# label_mapping = {0: "intent_positive", 1: "intent_negative", 2: "intent_neutral", 3: "intent_fallback", 4: "silence"}
# predicted_label = label_mapping.get(predicted_class, "Unknown")

# # 8. In kết quả dự đoán
# print(f"Input sentence: {sentence}")
# print(f"Predicted class ID: {predicted_class}")
# print(f"Predicted label: {predicted_label}")

# # Kết thúc phiên WandB
# wandb.finish()


### Tải và lưu tokenizer từ model gốc

In [ ]:
import os
import json
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import wandb

# 1. Tải mô hình từ artifact trên WandB
run = wandb.init(project="bert-intent-classification")  # Tên dự án trong WandB
artifact = run.use_artifact('doanngoccuong_nh/bert-intent-classification/last_model_epoch_30:v0', type='model')
artifact_dir = artifact.download()
print("Files in artifact_dir:", os.listdir(artifact_dir))

# Tải tokenizer từ mô hình gốc
original_tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

# Lưu các tệp cần thiết vào artifact_dir
original_tokenizer.save_pretrained(artifact_dir)

print(f"Tokenizer files saved to {artifact_dir}")

# 4. Tải mô hình đã lưu và tokenizer
model_path = artifact_dir  # Đường dẫn đến mô hình đã tải
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path)

# Chuyển mô hình sang chế độ đánh giá
model.eval()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

print(f"Model loaded and running on device: {device}")

# 5. Xử lý đầu vào
question = "What is the weather like today?"
answer = ""
inputs = tokenizer(
    sentence,
    return_tensors="pt",
    truncation=True,
    padding=True,
    max_length=512
)

# Chuyển đầu vào sang thiết bị phù hợp
inputs = {key: value.to(device) for key, value in inputs.items()}

# 6. Thực hiện dự đoán
with torch.no_grad():
    outputs = model(**inputs)  # Truyền đầu vào qua mô hình
    logits = outputs.logits  # Lấy logits từ đầu ra của mô hình
    predicted_class = torch.argmax(logits, dim=1).item()  # Lấy nhãn dự đoán

# 7. Mapping nhãn dự đoán sang tên nhãn
label_mapping = {0: "intent_positive", 1: "intent_negative", 2: "intent_neutral", 3: "intent_fallback", 4: "silence"}
predicted_label = label_mapping.get(predicted_class, "Unknown")

# 8. In kết quả dự đoán
print(f"Input sentence: {sentence}")
print(f"Predicted class ID: {predicted_class}")
print(f"Predicted label: {predicted_label}")

# Kết thúc phiên WandB
wandb.finish()
